In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
file1=pd.read_excel('D19_postSIEMBRA_p1_B1 BPs 100mW(000)_spike_list.xlsx')
#print(file1)
#print(type(file1))

In [ ]:
# LISTA DE VARIABLES
lista_pocillos = []
for letra in ['A', 'B', 'C', 'D']:
  for numero in range(1, 7):
    lista_pocillos.append(f"{letra}{numero}")

In [ ]:
file1_copy=file1.copy()

for pocillo in lista_pocillos:
  lista_electrodos = file1['Electrode']
  lista_electrodos_nuevos = [elec.replace(pocillo, '') for elec in lista_electrodos]
  file1['Electrode'] = lista_electrodos_nuevos


In [ ]:
(file_f2['Time (s)'] > timepoint) & (file_f2['Time (s)'] < (timepoint + window))

In [ ]:
from matplotlib.transforms import Bbox

def full_extent(ax, pad=0.0):
    """Get the full extent of an axes, including axes labels, tick labels, and
    titles."""
    # For text objects, we need to draw the figure first, otherwise the extents
    # are undefined.
    ax.figure.canvas.draw()
    items = ax.get_xticklabels() + ax.get_yticklabels()
#    items += [ax, ax.title, ax.xaxis.label, ax.yaxis.label]
    items += [ax, ax.title]
    bbox = Bbox.union([item.get_window_extent() for item in items])

    return bbox.expanded(1.0 + pad, 1.0 + pad)


def plot_raster(df, electrode, ax, fig):

  for repeticion in range(1, len(timepoints_start) + 1):
    df_plot = df[df['Repetition'] == repeticion]
    ax.scatter(df_plot['Time corrected'], [repeticion]*len(df_plot), marker='|', s=130, c='0', linewidths=0.5)

  ax.set_xlim(0, 15)
  ax.set_xticks(range(0, 16))

  ax.set_ylim(0, 12)

  ax.set_xlabel('Time (s)')
  ax.set_ylabel('Repetitions')
  plt.rcParams.update({'font.size': 10})
  ax.tick_params(axis='x', which='both', bottom=False, top=False)
  ax.tick_params(axis='y', which='both', left=False, right=False)
  ax.set_yticks(np.arange(1, 11, 1), np.arange(1, 11, 1))

  #add the ON/OFF states
  ax.plt([5, 10], [11.005, 11.005], 'r', lw=1, linestyle='--')
  ax.plt([0, 5], [11.005, 11.005], 'b', lw=1, linestyle='--')
  ax.plt([10, 15], [11.005, 11.005], 'b', lw=1, linestyle='--')

  ax.text(7.1, 11.2,'ON', color='red')
  ax.text(2, 11.2,'OFF', color='blue')
  ax.text(12, 11.2,'OFF', color='blue')



  extent = full_extent(ax).transformed(fig.dpi_scale_trans.inverted())
  fig.savefig(f'Electrode {electrode}.png', bbox_inches=extent)


def plot_bar(df, electrode, ax, fig, max_height=100):
  #plot for the average spike number of the OFF/ON/FF 5 sec sequence

  #make 3 lists with the off/on states
  list_off1, list_on, list_off2 = [],[],[]

  #create a data frame with the data
  for repeticion in range(1, len(timepoints_start) + 1):
      df_plot = df[df['Repetition'] == repeticion]
      list_off1.append(len(df_plot[df_plot['Time corrected'] < 5]))
      list_on.append(len(df_plot[(df_plot['Time corrected'] >= 5) & (df_plot['Time corrected'] < 10)]))
      list_off2.append(len(df_plot[df_plot['Time corrected'] >= 10]))

  #barplot
  ax.bar(0, np.mean(list_off1), yerr=np.std(list_off1), color='blue', capsize=5)
  ax.bar(1, np.mean(list_on), yerr=np.std(list_on), color='red', capsize=5)
  ax.bar(2, np.mean(list_off2), yerr=np.std(list_off2), color='blue', capsize=5)
  ax.set_xticks([0,1,2],['OFF', 'ON', 'OFF'])
  ax.set_yticks(np.arange(0, max_height+10, 10))
  ax.set_ylim(0,max_height)

  #create a data frame with the number of spikes in each repetition
  df_barplot = pd.DataFrame({'OFF 1': list_off1, 'ON': list_on, 'OFF 2': list_off2})

  #calculate the mean and the st deviation of the
  df_mean = df_barplot.mean()
  df_std = df_barplot.std()

  plot.ylabel('Number of spikes')
  plot.tick_params(axis='x', which='both', bottom=False, top=False)

  #save the barplot
  extent = full_extent(ax).transformed(fig.dpi_scale_trans.inverted())
  fig.savefig(f'Electrode {electrode}_barplot.png', bbox_inches=extent)

  #attach in the last two rows the values of the mean at st deviation
  df_barplot = df_barplot.append(df_mean, ignore_index=True)
  df_barplot = df_barplot.append(df_std, ignore_index=True)

  #chage the name to the rows of mean and st deviation
  index_df = df_barplot.index.tolist()
  index_df[-2:] = ['mean', 'std']
  df_barplot.index = index_df

  #save the dataframe to a csv
  df_barplot.to_csv(f'Electrode {electrode}.csv')
  #df_barplot

In [ ]:
df=file1_copy.copy()

#make files for each electrode

list_electrodes = ['11', '12', '13', '14',
                   '21', '22', '23', '24',
                   '31', '32', '33', '34',
                   '41', '42', '43', '44']

fig_raster, axs_raster = plot.subplots(4, 4, figsize=(4 * 6, 4 * 2.5))
fig_barplot, axs_barplot = plot.subplots(4, 4, figsize=(4 * 6, 4 * 2.5))


for idx_electrode, electrode in enumerate(list_electrodes):
  file_f2= df[df['Electrode'] == electrode]
  arr_reps = np.zeros(len(file_f2), dtype=int)
  times_corrected = np.zeros(len(file_f2))
  #print(file_f2)

  timepoints_start = [5, 20, 35, 50, 65, 80, 95, 110, 125, 140]
  window = 15

  # we want 10 repetitions of the measurement of 5 sec off, 5 sec on, 5 sec off
  #in the file we have 12 repetitions, so we are getting the info
  #r from sec 5 to sec 155, because at the beginning we have 10 sec off, and we only want 5

  #get the 10 data sets before making the scatter plot
  for repeticion, timepoint in enumerate(timepoints_start, 1):
    index_selection = (file_f2['Time (s)'] > timepoint) & (file_f2['Time (s)'] < (timepoint + window))
    arr_reps[index_selection] = repeticion
    times_corrected[index_selection] = file_f2.loc[index_selection, 'Time (s)'] - timepoint

  file_f2['Repetition'] = arr_reps
  file_f2['Time corrected'] = times_corrected



  # '12' -> 2, 2
  ax_y, ax_x = int(electrode[0]) - 1, 3 - (int(electrode[1]) - 1)
  plot_raster(df=file_f2, electrode=electrode, ax=axs_raster[ax_x, ax_y], fig=fig_raster)
  plot_bar(df=file_f2, electrode=electrode, ax=axs_barplot[ax_x, ax_y], fig=fig_barplot, max_height=80)